# Get fidelity

In [1]:
import os
import torch
import sys
sys.path.append('../../lib/exlib/src')

import sys
sys.path.append('../../src')
import sop

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sop.utils.seed_all(42)

# config
exp_config = sop.ImageNetConfig()
val_config = exp_config.get_config('val_sm')
val_config['evaluation']['batch_size'] = 2
val_config

/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/envs/rapids/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may hav

{'dataset': {'name': 'imagenet',
  'root': '/shared_data0/weiqiuy/datasets/imagenet'},
 'training': {'batch_size': 16,
  'num_epochs': 20,
  'mask_batch_size': 64,
  'optimizer': {'name': 'adamw', 'lr': 5e-06, 'weight_decay': 0.01}},
 'evaluation': {'split': 'val', 'num_data': 1, 'batch_size': 2},
 'model': {'type': 'vit',
  'base': 'google/vit-base-patch16-224',
  'sop': '/shared_data0/weiqiuy/sop/exps/imagenet_lr5e-06_tgtnnz0.2_gg0.0600_gs0.0100_ft_identify_fixk_scratch_ks3/best',
  'num_classes': 1000}}

In [2]:
# model
backbone_model, processor, backbone_config = sop.utils.imagenet_utils.get_model(val_config['model']['type'],
                                                                 backbone_model_name=val_config['model']['base'],
                                                                 backbone_processor_name=val_config['model']['base'],
                                                                )
backbone_model = backbone_model.to(device)

# get wrapped original model
from sop.utils.imagenet_utils import WrappedModel

original_model = WrappedModel(backbone_model, output_type='logits')
original_model = original_model.to(device)

# config
from exlib.modules.sop import SOPConfig, get_chained_attr

config = SOPConfig(os.path.join(val_config['model']['sop'], 'config.json'))

config.group_sel_scale = 0.05

config.__dict__.update(backbone_config.__dict__)
config.num_labels = len(backbone_config.id2label)

# get sop model
from sop.utils.imagenet_utils import get_model, get_wrapped_models

wrapped_backbone_model, class_weights, projection_layer = get_wrapped_models(
    backbone_model,
    config
)
wrapped_backbone_model = wrapped_backbone_model.to(device)
class_weights = class_weights.to(device)
projection_layer = projection_layer.to(device)

# sop
from exlib.modules.sop import SOPImageCls4

model = SOPImageCls4(config, wrapped_backbone_model, 
                     class_weights=class_weights, 
                     projection_layer=projection_layer)
state_dict = torch.load(os.path.join(val_config['model']['sop'], 
                                     'checkpoint.pth'))
print('Loaded step', state_dict['step'])
model.load_state_dict(state_dict['model'], strict=False)
model = model.to(device)
model.eval();

projection layer is not frozen
projection layer is not frozen
Loaded step 40100


In [3]:
# data

val_dataset, val_dataloader = sop.utils.get_dataset(val_config['dataset']['name'], 
                                          split=val_config['evaluation']['split'], 
                                          num_data=val_config['evaluation']['num_data'],
                                          batch_size=val_config['evaluation']['batch_size'],
                                          processor=processor)

Loaded 1000 images and 1000 classes


In [4]:
from sop.metrics import get_all_fidelity

In [5]:
# explainer_name = 'lime'
# fids = get_all_fidelity(val_dataloader, original_model, backbone_model, explainer_name, val_config['model']['num_classes'], device, skip=True)

In [6]:
import time

explainer_names = [
    # 'lime',
    # 'shap',
    # 'rise',
    # 'intgrad',
    # 'gradcam',
    # 'archipelago',
    # 'fullgrad',
    # 'attn', # need to make it an actual model
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet'
]

fids_dict = {}

for explainer_name in explainer_names:
    print(explainer_name)
    start = time.time()
    fids = get_all_fidelity(val_dataloader, original_model, backbone_model, explainer_name, 
                            val_config['model']['num_classes'], device, skip=True)
    end = time.time()
    fids_dict[explainer_name] = {
        'fid': fids,
        'time': end - start
    }

mfaba


  0%|          | 0/500 [00:00<?, ?it/s]

len(expln) 2
agi


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

len(expln) 2
xdnn


  0%|          | 0/500 [00:00<?, ?it/s]

attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 224])
outputs torch.Size([16, 1000])
attributions torch.Size([16, 3, 224, 2

  0%|          | 0/500 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 79.15 GiB total capacity; 66.00 GiB already allocated; 16.19 MiB free; 67.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
results_path = 'results/fidelity.pt'

torch.save(fids_dict, results_path)

# Results

In [ ]:
# need attn and sop
import os
import torch
import sys
sys.path.append('/shared_data0/weiqiuy/exlib/src')
from tqdm.auto import tqdm

# results_dir = '/shared_data0/weiqiuy/sop/results/imagenet_s'
results_dir = '/scratch/weiqiuy/sop/fidelity/imagenet_s'
num_examples = 50

explainer_names = [
    'lime',
    'shap',
    'rise',
    'intgrad',
    'gradcam',
    'archipelago',
    'fullgrad',
    # 'attn', # need to make it an actual model
    'mfaba',
    'agi',
    'ampe',
    'bcos',
    'xdnn',
    'bagnet'
]

fids_dict = {}
for explainer_name in tqdm(explainer_names):
    print(explainer_name)
    # break
    fids = []
    
    dirname = os.path.join(results_dir, explainer_name)
    for fi in tqdm(range(num_examples)):
        # break

        data = torch.load(os.path.join(dirname, f'{fi}.pt'))
        fids.append(data['fid'])
    fids_dict[explainer_name] = torch.stack(fids)

  0%|          | 0/13 [00:00<?, ?it/s]

lime


  0%|          | 0/50 [00:00<?, ?it/s]

shap


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
torch.save(fids_dict, 'fids_dict.pt')

In [4]:
data.keys()

dict_keys(['expln', 'probs', 'fid'])

In [6]:
data['fid']

tensor([2.2274], device='cuda:0')